In [1]:

import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re
import numpy as np

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2


# natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag_sents
from nltk.stem import WordNetLemmatizer

# SciKit-Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from skopt import gp_minimize, forest_minimize, dummy_minimize

import pandas as pd

train = pd.read_csv('train.csv')


test = pd.read_csv('test.csv')


#train



In [2]:
#preprocessing


comment_data = train

#clean
comment_data['prep'] = comment_data['comment'].str.replace(r'[^\w\s]+', '') # tira td que nao palavra e espaco em branco
comment_data['prep'] = comment_data['prep'].str.lower()
comment_data['prep'] = comment_data['prep'].str.replace('(\d+)', ' num ')
comment_data['prep'] = comment_data['prep'].str.replace(r'http(?<=http).*', ' ')
comment_data['prep'] = comment_data['prep'].str.replace(r'\s+', " ")
comment_data['prep'] = comment_data['prep'].str.replace(" +", " ")

#comment_data





test_data = test
#clean
test_data['prep'] = test_data['comment'].str.replace(r'[^\w\s]+', '')
test_data['prep'] = test_data['prep'].str.lower()
test_data['prep'] = test_data['prep'].str.replace('(\d+)', ' num ')
test_data['prep'] = test_data['prep'].str.replace(r'http(?<=http).*', ' ')
test_data['prep'] = test_data['prep'].str.replace(r'\s+', " ")
test_data['prep'] = test_data['prep'].str.replace(" +", " ")

In [3]:
lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])

comment_data['prep'] = comment_data['prep'].apply(lemmatize_col)
test_data['prep'] = test_data['prep'].apply(lemmatize_col)

# stopwords
stop = stopwords.words('english')
comment_data['prep'] = comment_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['prep'] = test_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [4]:
clean_data = comment_data['prep'].to_numpy()
clean_labels = comment_data['subreddit'].to_numpy()


In [10]:
#com pre proc

clean_data
clean_labels

clean_test=test_data['prep']
#features_train

### com preproc

In [5]:
#Decision Tree
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

param_dist = {'clf__criterion': ['gini'],
             'clf__min_samples_split': range(2,100,10)}



pipeline_DecisionTreeClassifier = Pipeline([
                     ('vect', CountVectorizer(ngram_range=(1,2), min_df = 10, max_df = 1.)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', DecisionTreeClassifier(random_state=0))
])

grid_search = GridSearchCV(pipeline_DecisionTreeClassifier, param_grid = param_dist, n_jobs=-1, verbose=1, cv=3, scoring='accuracy')
grid_search.fit(clean_data, clean_labels)
print(grid_search.best_score_)
print(grid_search.cv_results_)

previsoes_test_pre_proc=grid_search.predict(clean_test)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 10.4min finished


0.28099999999999997
{'mean_fit_time': array([109.53653725,  78.25729593,  70.16194574,  80.4855183 ,
        58.46328163,  60.53703634,  60.54092733,  61.12919172,
        90.86585164,  84.41392016]), 'std_fit_time': array([ 0.67646766, 17.79949636,  8.16119288,  0.67793147,  0.2942137 ,
        1.62034554,  0.29106425,  1.38394375,  1.22073467,  4.20848265]), 'mean_score_time': array([1.4823854 , 1.37764843, 1.47404965, 1.53144844, 1.42962011,
       1.41660412, 1.37138255, 2.0242219 , 2.19510587, 1.99014982]), 'std_score_time': array([0.1018871 , 0.02819107, 0.1103874 , 0.01523926, 0.06072799,
       0.09494082, 0.10242028, 0.10251352, 0.10093275, 0.65521162]), 'param_clf__criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'gini', 'gini', 'gini'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_clf__min_samples_sp

NameError: name 'clean_test' is not defined

In [12]:
'''anotar resultados 0.2675666666666667

cv=10 ...0.2770166666666667'''

'anotar resultados 0.2675666666666667\n\ncv=10 ...0.2770166666666667'

In [13]:
test

,id,comment,prep
0,0,Holy shit a shot counter.,think prestige point expire ever skin buy avai...
1,1,It doesn't matter that it isn't hard to rememb...,whats going happen refused asilum appeal
2,2,I find it funny that this is downvoted,anecdotal evidence anecdotal clearly everyone ...
3,3,They are really getting ridicoulous with all t...,look dude due respect music isnt people look l...
4,4,He's Eden's best friend,hope get doomhammer back
...,...,...,...
19995,19995,These officials are almost as incompetent as o...,see one frame smile num chilling many goosebump
19996,19996,honestly the Patriot act really fucked our com...,people spend time money movie dont want see ty...
19997,19997,My friend is now looking online for a thanos c...,said let watch
19998,19998,I really liked Thor Ragnarok and both Guardian...,im sure count second enterprise term battle star


In [14]:
test_data

,id,comment,prep
0,0,Holy shit a shot counter.,think prestige point expire ever skin buy avai...
1,1,It doesn't matter that it isn't hard to rememb...,whats going happen refused asilum appeal
2,2,I find it funny that this is downvoted,anecdotal evidence anecdotal clearly everyone ...
3,3,They are really getting ridicoulous with all t...,look dude due respect music isnt people look l...
4,4,He's Eden's best friend,hope get doomhammer back
...,...,...,...
19995,19995,These officials are almost as incompetent as o...,see one frame smile num chilling many goosebump
19996,19996,honestly the Patriot act really fucked our com...,people spend time money movie dont want see ty...
19997,19997,My friend is now looking online for a thanos c...,said let watch
19998,19998,I really liked Thor Ragnarok and both Guardian...,im sure count second enterprise term battle star


In [15]:
#SEM PRE PROC


train = pd.read_csv('train.csv')
#print(train)

test = pd.read_csv('test.csv')


features_train=train['comment']

target_train=train['subreddit']



features_test=test['comment']


features_train
#target_train

#test.head(20)

0        I think prestige points should not expire ever...
1        Whats going to happen with them if they will b...
2        Anecdotal evidence is anecdotal. Clearly by “e...
3        Look dude, with all due respect, your music is...
4                        Hope he gets the doomhammer back!
                               ...                        
59995                         Yo this guy Luka pretty good
59996                        Unplug these things right now
59997    Well said. Do you think they’ll resonate with ...
59998    So we can impeach a president for lying? Pleas...
59999    Too broad dude, get ready for the shit my pant...
Name: comment, Length: 60000, dtype: object

In [16]:
features_test

0                                Holy shit a shot counter.
1        It doesn't matter that it isn't hard to rememb...
2                   I find it funny that this is downvoted
3        They are really getting ridicoulous with all t...
4                                  He's Eden's best friend
                               ...                        
19995    These officials are almost as incompetent as o...
19996    honestly the Patriot act really fucked our com...
19997    My friend is now looking online for a thanos c...
19998    I really liked Thor Ragnarok and both Guardian...
19999                        last info changes everything.
Name: comment, Length: 20000, dtype: object

### sem preproc

In [17]:
#Decision Tree
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier


pipeline_DecisionTreeClassifier = Pipeline([
                     ('vect', CountVectorizer()),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', DecisionTreeClassifier(random_state=0))
])

grid_search = GridSearchCV(pipeline_DecisionTreeClassifier, {}, n_jobs=-1, verbose=1, cv=10, scoring='accuracy')
grid_search.fit(features_train, target_train)
print(grid_search.best_score_)
print(grid_search.cv_results_)

previsoes_test=grid_search.predict(features_test)



Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.4min finished


0.21385
{'mean_fit_time': array([115.29111938]), 'std_fit_time': array([26.91406565]), 'mean_score_time': array([0.47483046]), 'std_score_time': array([0.23469801]), 'params': [{}], 'split0_test_score': array([0.20516667]), 'split1_test_score': array([0.218]), 'split2_test_score': array([0.2145]), 'split3_test_score': array([0.21333333]), 'split4_test_score': array([0.2065]), 'split5_test_score': array([0.2185]), 'split6_test_score': array([0.21816667]), 'split7_test_score': array([0.21966667]), 'split8_test_score': array([0.20833333]), 'split9_test_score': array([0.21633333]), 'mean_test_score': array([0.21385]), 'std_test_score': array([0.00507962]), 'rank_test_score': array([1], dtype=int32)}


In [18]:
'''results  0.2060666666666667

results cv=5 ...0.20988333333333334

results cv=10 ...0.21385'''

'results  0.2060666666666667\n\nresults cv=5 ...0.20988333333333334\n\nresults cv=10 ...0.21385'

In [19]:
'''#gerar arquivo 

previsoes_test = pd.DataFrame(previsoes_test, columns=['Category'])

previsoes_test['Id'] = previsoes_test.index

previsoes_test = previsoes_test.reindex(columns=['Id', 'Category'])

previsoes_test.to_csv('decision_tree_17jul.csv', index=False)
'''



"#gerar arquivo \n\nprevisoes_test = pd.DataFrame(previsoes_test, columns=['Category'])\n\nprevisoes_test['Id'] = previsoes_test.index\n\nprevisoes_test = previsoes_test.reindex(columns=['Id', 'Category'])\n\nprevisoes_test.to_csv('decision_tree_17jul.csv', index=False)\n"